In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

In [ ]:
df = pd.read_csv('Chemical_Dataf.csv')

In [ ]:
features = ['Rate/kg', 'Rarity', 'Date Purchased','Amount Purchased']
target = 'Amount Purchased'

In [ ]:
df['Date Purchased'] = pd.to_datetime(df['Date Purchased'], format='%d-%m-%Y')

In [ ]:
df['Days Since Purchase'] = (df['Date Purchased'] - df['Date Purchased'].min()).dt.days

In [ ]:
X = df[features].values
y = df[target].values.reshape(-1, 1)

In [ ]:
X[:, 2] = df['Days Since Purchase'].astype(float)

In [ ]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

In [ ]:
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

In [ ]:
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i+sequence_length]
        sequences.append(seq)
    return np.array(sequences)

In [ ]:
sequence_length = 10
X_seq = create_sequences(X_scaled, sequence_length)
y_seq = create_sequences(y_scaled, sequence_length)[:, -1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)

Epoch 1/20
25/25 [==============================] - 0s 8ms/step - loss: 4.8620e-04
Epoch 2/20
25/25 [==============================] - 0s 8ms/step - loss: 2.4862e-04
Epoch 3/20
25/25 [==============================] - 0s 8ms/step - loss: 2.6440e-04
Epoch 4/20
25/25 [==============================] - 0s 8ms/step - loss: 2.2601e-04
Epoch 5/20
25/25 [==============================] - 0s 7ms/step - loss: 2.8890e-04
Epoch 6/20
25/25 [==============================] - 0s 7ms/step - loss: 2.8434e-04
Epoch 7/20
25/25 [==============================] - 0s 7ms/step - loss: 3.6499e-04
Epoch 8/20
25/25 [==============================] - 0s 7ms/step - loss: 2.6991e-04
Epoch 9/20
25/25 [==============================] - 0s 6ms/step - loss: 2.4540e-04
Epoch 10/20
25/25 [==============================] - 0s 8ms/step - loss: 1.6626e-04
Epoch 11/20
25/25 [==============================] - 0s 9ms/step - loss: 2.1616e-04
Epoch 12/20
25/25 [==============================] - 0s 10ms/step - loss: 3.0049e-04


In [ ]:
y_pred_scaled = model.predict(X_test)

7/7 [==============================] - 0s 3ms/step


In [ ]:
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_actual = scaler_y.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
mae = mean_absolute_error(y_actual, y_pred)
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 13.365937396733448


In [ ]:
mse = mean_squared_error(y_actual, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 296.50087773930983
